# Load packages and data 

In [57]:
import numpy as np 
import pylab as pl 
import dataretrieval.nwis as nwis
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

# Prepare final usgs gauges

In [74]:
usgs_linked = pd.read_csv('../local_data/tables/usgs_temporal_linked_w_trimmed.csv')
usgs_linked.set_index('STAID', inplace = True)
usgs_final = pd.read_csv('../local_data/tables/usgs_final_list.txt')
usgs_final.set_index('usgs_id', inplace=True)

## Remove gauges that must not be present 

In [75]:
idx_remove = usgs_final.loc[usgs_final.status == 'REMOVE'].index
usgs_linked.drop(idx_remove, inplace=True)

## Add gauges 

### Get the indformation of the gauges to add 

In [76]:
idx_add = usgs_final.loc[usgs_final.status == 'ADD'].index

In [82]:
sites = ['0%d' % i for i in idx_add]
info = nwis.get_info(sites = sites)#'0%d'%idx_add[0])
new_gauges = info[0][['site_no','dec_lat_va', 'dec_long_va','drain_area_va', 'station_nm' ]]

In [83]:
new_gauges.rename(columns={'station_nm':'STANAME', 'drain_area_va':'DRAIN_SQKM',
                          'dec_long_va':'lon','dec_lat_va':'lat'}, inplace = True)
new_gauges['DRAIN_SQKM'] = new_gauges['DRAIN_SQKM']*2.5899
new_gauges.set_index('site_no', inplace = True)
new_gauges['LINKNO'] = -9999

### Get the lat long of the previous gauges 

In [79]:
sites = ['0%d' % i for i in usgs_linked.index]
info = nwis.get_info(sites = sites)#'0%d'%idx_add[0])
usgs_linked_coord = info[0][['site_no','dec_lat_va', 'dec_long_va',]]
usgs_linked_coord.set_index('site_no', inplace = True)
usgs_linked.index = sites
usgs_linked['lat'] = usgs_linked_coord.loc[usgs_linked.index, 'dec_lat_va']
usgs_linked['lon'] = usgs_linked_coord.loc[usgs_linked.index, 'dec_long_va']

In [87]:
usgs_final = pd.concat([usgs_linked, new_gauges])

In [89]:
usgs_final.reset_index().to_csv('../local_data/tables/usgs_final_list.csv')